# RUN IN JULIA

Things I did:
1. Add new flow constraint such that in = out for each truck, demand location
2. Add MTZ constraints to avoid cycles within demand locations

Things to try speeding up:
1. Redundant constraints?
2. Better multi-objective form? See the slides... 
3. Put a limit on total number of vehicles
4. Only add feasible arcs? I.e. threshold against edges that are too big of a time in our dataset
5. OPtimize for less things? Don't need hourly wages?

In [1]:
# import Pkg
# Pkg.add("HDF5")

In [2]:
using JuMP
using Gurobi
using CSV
using DataFrames
using HDF5
using Base.Threads


In [ ]:
Tx = CSV.read("inputs/Tx.csv", DataFrame, drop=1:1) |> Matrix #N, N
Ty = CSV.read("inputs/Ty.csv",DataFrame, drop=1:1)|> Matrix; #M, N
Tz = CSV.read("inputs/Tz.csv",DataFrame, drop=1:1)|> Matrix; #N, M

M,N = size(Ty)
Cf = 1
Cw = 1
Ct = 3
alpha = 0.5
println("M: ", M)
println("N: ", N)

M: 370
N: 30


In [4]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 240)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-05
Set parameter TimeLimit to value 240


In [ ]:

@variable(model, o[1:M], Bin)
@variable(model, x[1:N, 1:N, 1:N], Bin)
@variable(model, y[1:N, 1:M, 1:N], Bin)
@variable(model, z[1:N, 1:N, 1:M], Bin)
@variable(model, f[1:N] >= 0)
@variable(model, u[1:N, 1:N]) #k, j
@variable(model, L >= 0)

@objective(model, Min, alpha*(Cf*sum(o[i] for i in 1:M) + 
Ct * sum(y[k, i, j] for k in 1:N, i in 1:M, j in 1:N) + 
Cw * sum(sum(y[k,i,j]*Ty[i,j] + z[k,j,i]*Tz[j,i] for i in 1:M, j in 1:N) + sum(x[k, j1, j2] * Tx[j1, j2] for j1 in 1:N, j2 in 1:N) for k in 1:N)) +
(1 - alpha) * L)


#MTZ Constraitns
for k in 1:N
    for j in 1:N
        @constraint(model, u[k, j] <= N)
        @constraint(model, u[k, j] >= 1)
    end
end

#(MTZ) Between Demand Locations
for j1 in 1:N #loc 1
    for j2 in 1:N #loc 2
        if j1 != j2
            for k in 1:N
                @constraint(model, u[k, j2] - u[k, j1] >= 1 - N*(1-x[k,j1,j2]))
            end
        end
    end
end

# Optional additional constriant
for j in 1:N
    for k in 1:N
        @constraint(model, x[k,j,j] == 0)
    end
end
# end additional constraint

for k in 1:N
    @constraint(model, sum(y[k,i,j] for i in 1:M, j in 1:N) <= 1)
end

for k in 1:N
    @constraint(model, sum(x[k,j1,j2] for j1 in 1:N, j2 in 1:N) <= (N - 1) * sum(y[k,i,j] for i in 1:M, j in 1:N))
end

for i in 1:M
    @constraint(model, sum(y[k,i,j] for k in 1:N, j in 1:N) <= N * o[i])
end

for k in 1:N
    for i in 1:M
        @constraint(model, sum(z[k,j,i] for j in 1:N) == sum(y[k,i,j] for j in 1:N))
    end
end

#FLOW CONSTRAINTS
#SUM
for j2 in 1:N 
    @constraint(model, sum(sum(y[k,i,j2] for i in 1:M) + sum(x[k,j1,j2] for j1 in 1:N) for k in 1:N) == 1)
end 

for j1 in 1:N 
    @constraint(model, sum(sum(x[k,j1,j2] for j2 in 1:N) + sum(z[k,j1,i] for i in 1:M) for k in 1:N) == 1)
end 

for j in 1:N
    for k in 1:N
        @constraint(model, sum(y[k,i,j] for i in 1:M) + sum(x[k,j1,j] for j1 in 1:N)  == 
        sum(x[k,j,j2] for j2 in 1:N) + sum(z[k,j,i] for i in 1:M) )
    end
end


# Now time constraints
for k in 1:N 
    @constraint(model, f[k] == sum(y[k,i,j] * Ty[i,j] for i in 1:M, j in 1:N) + sum(x[k,j1,j2] * Tx[j1,j2] for j1 in 1:N, j2 in 1:N))
end

for k in 1:N 
    @constraint(model, f[k] <= L)
end

In [6]:
optimize!(model)

Set parameter TimeLimit to value 240
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 41350 rows, 694301 columns and 3570760 nonzeros
Model fingerprint: 0x8ec9fa06
Variable types: 931 continuous, 693370 integer (693370 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e-02, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 2730 rows and 900 columns
Presolve time: 3.89s
Presolved: 38620 rows, 693401 columns, 3232360 nonzeros
Variable types: 900 continuous, 692501 integer (692500 binary)
Found heuristic solution: objective 77.0000000
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.02s

Barrier performed 0 iterations in 6.23 seconds (15.56 work units)
Barrier s

In [8]:
try
    rm("results/output.h5")
catch
    1
end
h5write("results/output.h5", "factories", value.(o))
h5write("results/output.h5", "x_edges", value.(x))
h5write("results/output.h5", "y_edges", value.(y))
h5write("results/output.h5", "z_edges", value.(z))
